In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import psutil

In [2]:
from pandas.tseries.offsets import Day, Week

In [3]:
psutil.virtual_memory().available >> 30

146

In [4]:
data1 = pd.read_parquet('/main/data/MVideo_2/temp_5_1.parquet')

In [5]:
data = data1[['DATE', 'PRODUCT_ID', 'STORE_ID', 'sales.SUM(trans.UNITS_SOLD)_W0', 'sales.SUM(trans.REVENUE_INCL_TAX)_W0',
              'REGULAR_SALES_PRICE_INCL_TAX']].copy()

In [6]:
del data1

In [7]:
psutil.virtual_memory().available >> 30

146

In [10]:
data['sales.SUM(trans.UNITS_SOLD)_W0'] = np.clip(data['sales.SUM(trans.UNITS_SOLD)_W0'], 0, None)

In [12]:
data['sales.SUM(trans.REVENUE_INCL_TAX)_W0'] = np.clip(data['sales.SUM(trans.REVENUE_INCL_TAX)_W0'], 0, None)

In [13]:
data = data.sort_values(['STORE_ID', 'PRODUCT_ID', 'DATE']).reset_index(drop = True)

In [14]:
data['A_PRICE'] = data['sales.SUM(trans.REVENUE_INCL_TAX)_W0'] / data['sales.SUM(trans.UNITS_SOLD)_W0']

In [15]:
data.A_PRICE = data.groupby(['PRODUCT_ID', 'STORE_ID']).A_PRICE.fillna(method = 'ffill')

In [18]:
tdata = data[data.DATE < '2018-03-18'].copy()

In [19]:
tdata.A_PRICE.isnull().sum() / tdata.shape[0]

0.08832982345730221

In [20]:
tdata.A_PRICE = data.groupby(['PRODUCT_ID', 'STORE_ID']).A_PRICE.fillna(method = 'bfill')

In [21]:
tdata.A_PRICE.isnull().sum() / tdata.shape[0]

0.027034599272559613

In [22]:
tdata.rename(columns = {'A_PRICE': 'PRICE'}, inplace = True)

In [23]:
tdata.drop(columns = ['sales.SUM(trans.UNITS_SOLD)_W0', 'sales.SUM(trans.REVENUE_INCL_TAX)_W0', 'REGULAR_SALES_PRICE_INCL_TAX'],
           inplace = True)

In [24]:
data = data.merge(tdata, how = 'left').sort_values(['STORE_ID', 'PRODUCT_ID', 'DATE'])

In [25]:
data.PRICE.fillna(data.A_PRICE, inplace = True)

In [26]:
data.PRICE.fillna(data.REGULAR_SALES_PRICE_INCL_TAX, inplace = True)

In [27]:
data.drop(columns = ['REGULAR_SALES_PRICE_INCL_TAX', 'A_PRICE'], inplace = True)

In [28]:
#data.to_parquet('/main/data/MVideo_2/temp_5_2.parquet')

In [29]:
tdata = data.groupby(['PRODUCT_ID', 'STORE_ID'])[['DATE']].min().reset_index()

In [30]:
tdata['START'] = 1

In [31]:
data = data.merge(tdata, how = 'left').sort_values(['STORE_ID', 'PRODUCT_ID', 'DATE'])

In [32]:
data['SALES_BIN'] = data['sales.SUM(trans.UNITS_SOLD)_W0'] > 0

In [33]:
start = data.START.values

In [34]:
sls = data.SALES_BIN.astype('int').values

In [35]:
sls[0] == 0

True

In [36]:
wss = []

In [37]:
for i in range(data.shape[0]):
    if sls[i] == 1:
        t = 0
    elif start[i] != 1:
        t = wss[i - 1] + 1
    else:
        t = 1
    wss.append(t)

In [38]:
data['WEEK_SINCE_SALE'] = wss

In [39]:
sw = []

In [40]:
for i in range(data.shape[0]):
    if start[i] == 1:
        t = sls[i]
    else:
        t = sw[i - 1] + sls[i]
    sw.append(t)

In [41]:
sr = []

In [42]:
for i in range(data.shape[0]):
    if start[i] == 1:
        t = 1
    else:
        t = sr[i - 1] + 1
    sr.append(t)

In [43]:
data['SALES_WEEKS'] = sw

In [44]:
data['SALES_RANGE'] = sr

In [45]:
data['SALE_FREQUENCY'] = data.SALES_WEEKS / data.SALES_RANGE

In [46]:
qsls = data['sales.SUM(trans.UNITS_SOLD)_W0'].values

In [47]:
ss = []

In [48]:
for i in range(data.shape[0]):
    if start[i] == 1:
        t = qsls[i]
    else:
        t = ss[i - 1] + qsls[i]
    ss.append(t)

In [49]:
data['SALES_SUM'] = ss

In [50]:
data['SOLD_AVERAGE'] = data.SALES_SUM / data.SALES_RANGE

In [51]:
#data.to_parquet('/main/data/MVideo_2/temp_5_2.parquet')

In [52]:
#data = pd.read_parquet('/main/data/MVideo_2/temp_5_2.parquet')

In [53]:
data.shape[0]

2831698

In [54]:
tdata = data[['PRODUCT_ID', 'DATE']].drop_duplicates().sort_values(['PRODUCT_ID', 'DATE']).reset_index(drop = True)

In [55]:
tdata['NEWNESS'] = tdata.groupby('PRODUCT_ID').cumcount()

In [56]:
data = data.merge(tdata, how = 'left').sort_values(['STORE_ID', 'PRODUCT_ID', 'DATE'])

In [57]:
esls1 = []
ewss1 = []
sba1 = []

In [58]:
a = 0.5

In [59]:
for i in range(data.shape[0]):
    if start[i] == 1:
        if qsls[i] == 0:
            z = 1
            v = 2
            y = 0.5
        else:
            z = qsls[i]
            v = 1
            y = qsls[i]
    else:
        if qsls[i] == 0:
            z = esls1[i - 1]
            v = ewss1[i - 1]
            y = sba1[i - 1]
        else:
            z = a * qsls[i] + (1 - a) * esls1[i - 1]
            v = a * (wss[i] + 1) + (1 - a) * ewss1[i - 1]
            y = z / v
    esls1.append(z)
    ewss1.append(v)
    sba1.append(y)

In [60]:
data['DEMAND_ESTIMATE_50'] = esls1

In [61]:
data['INTERVAL_ESTIMATE_50'] = ewss1

In [62]:
data['SBA_50'] = sba1

In [63]:
data.SBA_50 *= (1 - a / 2)

In [64]:
esls2 = []
ewss2 = []
sba2 = []

In [65]:
a = 0.95

In [66]:
for i in range(data.shape[0]):
    if start[i] == 1:
        if qsls[i] == 0:
            z = 1
            v = 2
            y = 0.5
        else:
            z = qsls[i]
            v = 1
            y = qsls[i]
    else:
        if qsls[i] == 0:
            z = esls2[i - 1]
            v = ewss2[i - 1]
            y = sba2[i - 1]
        else:
            z = a * qsls[i] + (1 - a) * esls2[i - 1]
            v = a * (wss[i] + 1) + (1 - a) * ewss2[i - 1]
            y = z / v
    esls2.append(z)
    ewss2.append(v)
    sba2.append(y)

In [67]:
data['DEMAND_ESTIMATE_95'] = esls2

In [68]:
data['INTERVAL_ESTIMATE_95'] = ewss2

In [69]:
data['SBA_95'] = sba2

In [70]:
data.SBA_95 *= (1 - a / 2)

In [71]:
#data.to_parquet('/main/data/MVideo_2/temp_5_2.parquet')

In [72]:
data.drop(columns = ['sales.SUM(trans.UNITS_SOLD)_W0', 'sales.SUM(trans.REVENUE_INCL_TAX)_W0', 'START', 'SALES_BIN',
                     'SALES_WEEKS', 'SALES_SUM'], inplace = True)

In [73]:
data[['WEEK_SINCE_SALE', 'SALES_RANGE', 'NEWNESS']] = data[['WEEK_SINCE_SALE', 'SALES_RANGE', 'NEWNESS']].astype('int32')

In [74]:
data[['SALE_FREQUENCY', 'SOLD_AVERAGE', 'DEMAND_ESTIMATE_50', 'INTERVAL_ESTIMATE_50', 'SBA_50', 'DEMAND_ESTIMATE_95',
      'INTERVAL_ESTIMATE_95', 'SBA_95']] = data[['SALE_FREQUENCY', 'SOLD_AVERAGE', 'DEMAND_ESTIMATE_50',
                                                 'INTERVAL_ESTIMATE_50', 'SBA_50', 'DEMAND_ESTIMATE_95',
                                                 'INTERVAL_ESTIMATE_95', 'SBA_95']].astype('float32')

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2831698 entries, 0 to 2831697
Data columns (total 15 columns):
DATE                    datetime64[ns]
PRODUCT_ID              int32
STORE_ID                int32
PRICE                   float32
WEEK_SINCE_SALE         int32
SALES_RANGE             int32
SALE_FREQUENCY          float32
SOLD_AVERAGE            float32
NEWNESS                 int32
DEMAND_ESTIMATE_50      float32
INTERVAL_ESTIMATE_50    float32
SBA_50                  float32
DEMAND_ESTIMATE_95      float32
INTERVAL_ESTIMATE_95    float32
SBA_95                  float32
dtypes: datetime64[ns](1), float32(9), int32(5)
memory usage: 194.4 MB


In [77]:
data1 = pd.read_parquet('/main/data/MVideo_2/dataset_main_5_0.parquet')

In [79]:
data1 = data1.merge(data)

In [80]:
data1.info(max_cols = 999)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2831698 entries, 0 to 2831697
Data columns (total 127 columns):
DATE                                                datetime64[ns]
INTERNET                                            int32
MAINCATEGORY_ID                                     int32
MONTH(DATE)                                         int32
PRODUCT_ID                                          int32
PROMOTIONS_AVAILABLE_ABS_DIFF_MEAN_8                float32
PROMOTIONS_AVAILABLE_DIFF_P1                        int32
PROMOTIONS_AVAILABLE_P1                             int32
PROMOTIONS_AVAILABLE_W0                             int32
PROMOTIONS_AVAILABLE_W1                             int32
PROMOTIONS_AVAILABLE_W2                             int32
PROMOTIONS_AVAILABLE_W3                             int32
PROMOTIONS_AVAILABLE_W4                             int32
PROMOTIONS_AVAILABLE_W5                             int32
PROMOTIONS_AVAILABLE_W6                             int32
PROMO

In [82]:
data1.to_parquet('/main/data/MVideo_2/dataset_main_5_1.parquet')